In [12]:
import pandas as pd
import sqlalchemy as sql
import datetime
import pymysql

# read password and user to database
credentials_file = r'./credentials.pw'

user = 'doseReader'
pw = 'heiko'

host="twofast-rpi3-0"  # your host
user=user # username
passwd=pw  # password
db="NG_twofast_DB" # name of the database
connect_string = 'mysql+pymysql://%(user)s:%(pw)s@%(host)s:3306/%(db)s'% {"user": user, "pw": pw, "host": host, "db": db}
sql_engine = sql.create_engine(connect_string)

def retrieveLiveData(sql_engine, pastSeconds=7200):  # read past 2hrs by default
    query = """SELECT * FROM HBox_Uno ORDER BY id DESC LIMIT {}""".format(pastSeconds)
    df_dose = pd.read_sql(query, sql_engine)
    # voltage_dose, counts_WS, counts_BS, counts_DF
#     df_dose.rename(columns={0: 'ID', 1: 'date', 2: 'dose_voltage', 3: 'HV_current', 4: 'HV_voltage'}, inplace=True)
#     df_dose = df_dose.set_index(['ID'])
    
    df_dose['dose'] = df_dose['dose_voltage'] * 3000 / 5.5  # conversion factors according to current output signal muSv/hr
    df_dose['date'] = df_dose['time']
    
    df_dose = df_dose[['date', 'dose', 'HV_current', 'HV_voltage']]
    return df_dose

df = retrieveLiveData(sql_engine, 7200)
df.head()

,date,dose,HV_current,HV_voltage
0,2019-08-26 18:49:48.556600,49.090909,0.0,0.0
1,2019-08-26 18:49:47.557112,49.090909,0.0,0.0
2,2019-08-26 18:49:46.553979,49.090909,0.0,0.0
3,2019-08-26 18:49:45.554362,49.090909,0.0,0.0
4,2019-08-26 18:49:44.551301,49.090909,0.0,0.0
